# Demand Estimation Exercise

In this assignment, you will learn how to simulate market data and estimate simple random coefficient demand models. The assignment should serve as an introduction to the structural estimation in heterogenous products demand models.

Let's first define the model.

todo: Here you need to insert all the model definiton and reference to the slides


$$ u_{ijt} = \beta_{it}^{'} x_j + \alpha_{it}p_{jt} + \xi_{jt} + \varepsilon_{ijt}$$

Where: 

$$ \alpha_{it} =  \mu_{\alpha} + \sigma_{\alpha} \nu_{it}^{\alpha} $$ 
$$ \beta_{it} =  \mu_{\beta} + \sigma_{\beta} \nu_{it}^{\beta} $$ 

And $\nu_{it}^{\alpha}$ and $\nu_{it}^{\beta}$ are standard normally distributed. 

TO DO: Define all the other parameters and how they are constructed in the simulation for the students 


decision to make: 
                - should they work only in pandas or in numpy?

                  - should they simulate data themselves? or just import my module?

                  - should they write the GMM estimator themselves or just apply pyBLP more times until convergence? 
                   
                  - maybe it is enough for them to write the strucural function parameters? 

                 


### Step 1: Simulate Data



In [ ]:
import market
import numpy as np 
import pandas as pd 
import statsmodels.api as sm
import scipy.stats as stats
import matplotlib.pyplot as plt
import pyblp

The first step is to initialize the model class

To do this you need to import the "simulation" module that was especially constructed for this course. Make sure the assignment notebook and the simulate are in the same folder if you have not prespecified a different path for the module.  (FIND A WAY FOR THE STUDENTS TO NOT RUN IT LOCAL, BUT ON THE GIT SERVER SUCH THAT THEY DO NOT NEED TO HAVE ANYTHING INSTALLED)

Lets start by setting the parameters defining the market. These include the number of firms `n_firms`, number of consumers `n_cons`, number of product charactistics `n_char` and the number of time periods (or separated markets) `T`. The larger numbers you choose the longer the opimization will take. 

In [ ]:
# Set the number of firms and consumers, the number of characteristics and the
# number of time steps.
n_firms = 10
n_cons = 1000
n_char = 2
T = 100

# Initialize the market object
market_object = market.MarketData(n_firms, n_cons, n_char, T)

We have now created an _object_, which contains all the data and functions used to generate the data. The `market.py` file contains the Python code used to define this object. The simulation code makes use of the Object-Oriented Programming (OOP) paradigm. You do not need to change this code, but you can give it a look and try to understand the simulation process. 

We can now print the object, which will give us a summary of the main parameters of our simulation:

In [ ]:
print(market_object)

In this exercise we are going to try and estimate the parameters underlying the indirect mean utility of the consumers for buying a certain product (again reference to the slide and certain). You can easily retrive the true parameters by running the code below.
THESE ARE THE PARAMETERS WE ARE GOING TO TRY AND ESTIMATE

In [ ]:
# For the parameters underlying the distribution of the price coefficients 
print(f"This is the mean price coefficeient {market_object.alpha_mean}")
print(f"This is the standard deviation of the price coefficient {market_object.alpha_sd}")

# For the parameters underlying the distribution of the product characteristics coefficients 
print(f"These are the means characteristics coefficeients \n{market_object.beta_0}")
print(f"These are standard deviations of the characteritics coefficients \n {market_object.beta_sd}")


### PART 1. Data exploration


Exercise 1: Look into the data characterizing you market. Print the dataframe object. Describe what you see. Is it all realistic? Which values are you likely to not have as an econometrician working on real datasets?

In [ ]:
df = market_object.generate_simulated_data()

# Print here the dataframe (delete answer before hand in students)
print(df)

# Describe in 100 words max what you see (as a comment in the cell or in a 
# separate markdown cell: 
# Which values are you not likely to not observe when working with real datasets?

Exercise 2: Create a histogram with: 

1. The distribution of prices 
2. The distribution of market shares 

You should use the columns from the generated dataframe. 

In [ ]:
# Create a histogram
plt.hist(df["prices"], bins=50, edgecolor='black')

# Set labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title(' Prices Histogram')

# Display the plot
plt.show()

In [ ]:
# Create a histogram
plt.hist(df["shares"], bins=50, edgecolor='black')

# Set labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Market Shares Histogram')

# Display the plot
plt.show()

### Part 2: Simple Logit Estimation

Exercise 1: Estimate the simple logit model disregarding consumer heterogeneity

Trying to estimate with the logit function: 

$$ ln(s_0) - ln(s_n) =   \beta^{'} x_j + \alpha p_{jt}  $$

Check if this estimation leads to the correct coefficients and what we can do to improve it. 

In [ ]:
# Maybe let the students do this in pandas because it seems to much to ask to 
# use the attributes in 1-2 weeks? 
reshape = np.reshape(market_object.market_shares, (T, n_firms))
sum_market_shares = np.sum(reshape, axis=1)
repeated_sum_shares = np.reshape(np.repeat(sum_market_shares, n_firms), (n_firms*T, 1))
y  = np.log(market_object.market_shares) - np.log(repeated_sum_shares)    

In [ ]:
x = df[['prices','char1', 'char2']]
# with sklearn
mod = sm.OLS(y, x).fit()
print(mod.summary()) 

Exercise 2: Compare the estimates with the true estimates? What is wrong here? 


In [ ]:
print(f"The true parameters are alpha = {market_object.alpha_mean} and beta = \n {market_object.beta_0} ")



print(f"The estimates parameters are alpha = {mod.params[0]} and beta = \n {mod.params[1:n_char+1]}")

In [ ]:
# Additional visualization students do not need to do this but it is good for 
# visualization

fig, axes = plt.subplots(2, 1, figsize=(6, 4))

for i,ax in enumerate(axes.flatten()):
    ax.axvline(market_object.beta_0[i], label="true", color="blue")

    # Plot (distribution of) estimated coefficient
    mu = mod.params[i+1]
    sigma = mod.bse[i+1]
    x1 = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
    x2 = np.linspace(mu - 1.96*sigma, mu + 1.96*sigma, 1000)
    
    ax.axvline(mu, label="estimated", color="red")
    ax.plot(x1, stats.norm.pdf(x1, mu, sigma), color="red")
    ax.fill_between(x2, stats.norm.pdf(x2, mu, sigma), color="red", alpha=0.4)
    
    ax.set_title(f"$\\beta_{i+1}$")

plt.tight_layout()
plt.show()

### Part 3: Estimation with the PyBLP package (without supply side problem)


In [ ]:
X1_formulation = pyblp.Formulation('0 + prices + char1 + char2')
X2_formulation = pyblp.Formulation('0 + prices + char1 + char2')
product_formulations = (X1_formulation, X2_formulation)
product_formulations

In [ ]:
mc_integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed': 0})
mc_integration

In [ ]:
mc_problem = pyblp.Problem(product_formulations, df, integration=mc_integration)


In [ ]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [ ]:
results1 = mc_problem.solve(sigma=np.ones((3, 3)), optimization=bfgs)



### Part 4 - Here either writing linear estimation themselves or put in the supply side cost structure and disable optimal instruments

In [ ]:
product_data = df
# Setting up the formulations
product_formulations = (pyblp.Formulation('0 + prices + char1 + char2'), pyblp.Formulation('0 + prices + char1+ char2'),pyblp.Formulation('cost'))

mc_integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed': 0})
problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-6})
estimation = problem.solve(sigma=(np.ones((3, 3))/2),beta=[1., 1., 1.], optimization=bfgs)